In [1]:
import pandas as pd
import akshare as ak
import talib

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="300750", period="daily", start_date="19900101", end_date='21000101', adjust="qfq")
fund_etf_hist_em_df = ak.fund_etf_hist_em(symbol="159915", period="daily", start_date="20000101", end_date="21000101", adjust="qfq")
stock_zh_a_hist_df_s=stock_zh_a_hist_df[stock_zh_a_hist_df["日期"]>pd.to_datetime("2021-01-01")]
fund_etf_hist_em_df_s = fund_etf_hist_em_df[pd.to_datetime(fund_etf_hist_em_df["日期"])>pd.to_datetime("2021-01-01")]

fund_etf_hist_em_df_s["日期"] = pd.to_datetime(fund_etf_hist_em_df_s["日期"])
stock_zh_a_hist_df_s["日期"] = pd.to_datetime(stock_zh_a_hist_df_s["日期"])
merge_stock_zh_df = pd.merge(stock_zh_a_hist_df_s[["日期", "涨跌幅"]], fund_etf_hist_em_df_s[["日期", "涨跌幅"]], how="inner", on="日期")

# 填充缺失值为上一条记录的涨跌幅
merge_stock_zh_df["涨跌幅_x"].fillna(method="ffill", inplace=True)
merge_stock_zh_df["涨跌幅_y"].fillna(method="ffill", inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3380\661919299.py:7: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  stock_zh_a_hist_df_s=stock_zh_a_hist_df[stock_zh_a_hist_df["日期"]>pd.to_datetime("2021-01-01")]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3380\661919299.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund_etf_hist_em_df_s["日期"] = pd.to_datetime(fund_etf_hist_em_df_s["日期"])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3380\661919299.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [3]:
merge_stock_zh_df

,日期,涨跌幅_x,涨跌幅_y
0,2021-01-04,15.65,3.84
1,2021-01-05,-0.54,0.40
2,2021-01-06,1.22,0.50
3,2021-01-07,1.64,1.76
4,2021-01-08,-2.18,-0.65
...,...,...,...
847,2024-07-05,-1.10,0.43
848,2024-07-08,-0.96,-1.66
849,2024-07-09,-0.08,1.50
850,2024-07-10,-1.55,-0.12


In [10]:
beta = talib.BETA(merge_stock_zh_df["涨跌幅_x"], merge_stock_zh_df["涨跌幅_y"], timeperiod=30)
beta

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
847   -0.000901
848   -0.000433
849   -0.000356
850   -0.000575
851    0.003025
Length: 852, dtype: float64

In [13]:
beta_df = pd.DataFrame(beta)
beta_df["日期"]=merge_stock_zh_df["日期"]
merge_stock_zh_df_1 = pd.merge(merge_stock_zh_df, beta_df, how="inner")
merge_stock_zh_df_1

,日期,涨跌幅_x,涨跌幅_y,0
0,2021-01-04,15.65,3.84,NaN
1,2021-01-05,-0.54,0.40,NaN
2,2021-01-06,1.22,0.50,NaN
3,2021-01-07,1.64,1.76,NaN
4,2021-01-08,-2.18,-0.65,NaN
...,...,...,...,...
847,2024-07-05,-1.10,0.43,-0.000901
848,2024-07-08,-0.96,-1.66,-0.000433
849,2024-07-09,-0.08,1.50,-0.000356
850,2024-07-10,-1.55,-0.12,-0.000575
